In [1]:
#!pip install scikit-learn

In [27]:
#!pip install numba

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 6.1 MB/s eta 0:00:000:00:01m eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 MB 7.7 MB/s eta 0:00:00m eta 0:00:010:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 9.5 MB/s eta 0:00:00m eta 0:00:010:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.1
    Uninstalling numpy-2.1.1:
      Successfully uninstalled numpy-2.1.1


In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
from sklearn.preprocessing import TargetEncoder
import gc
import numba


In [2]:
#a = pd.DataFrame({'a':[1,2,3], 'b':[3,4,5], 'c':[7,8,9]})
#a

In [3]:
#a = pd.DataFrame({'a':[1,2,3], 'b':[3,4,5], 'c':[7,8,9]})
#a = a[['c','b','a']]
#a.ewm(alpha=0.7, min_periods=0, axis=1).mean(engine="numba")

In [4]:
import sys

def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)
    
   

In [5]:
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                              locals().items())), key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

                 TargetEncoder:  1.7 KiB
                            _i:  384.0 B
                           _i4:  384.0 B
                           _i5:  260.0 B
                           _ii:  183.0 B
                           _i3:  183.0 B
                           _i1:  173.0 B
                          open:  160.0 B
                    sizeof_fmt:  160.0 B
                           _ih:  120.0 B


In [6]:
#numba.set_num_threads(5)

In [7]:
#z=a.T.ewm(alpha=0.7, min_periods=0).mean(engine="numba").iloc[:,-1];

In [8]:
data_train = pd.read_csv('../data_cleaned/data_train.csv')
item_categories = pd.read_csv('../data_cleaned/item_categories.csv')
items = pd.read_csv('../data_cleaned/items.csv')
shops = pd.read_csv('../data_cleaned/shops.csv')

In [9]:
#merged = data_train.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')

In [10]:
def parse_city(shop_name):
    if shop_name.split()[0] == '!Якутск':
        return  'Якутск'

    if shop_name.split()[0] == 'Сергиев':
            return  'Сергиев Посад'
    else:
        return shop_name.split()[0]

In [11]:
#dates = pd.to_datetime(data_train['date'], format='%d.%m.%Y')


In [12]:
import calendar

def number_to_month(numbers):
    # Create a list of month names using the calendar module
    month_names = list(calendar.month_name)[1:]  # Extract months from 1 to 12 (exclude the empty element at index 0)
    
    # Convert the numbers to month names
    result = [month_names[num % 12] for num in numbers]  # Use num % 12 to handle numbers beyond 11
    
    return result


In [13]:

def number_to_month(numbers):
    result = [num %12 for num in numbers]  # Use num % 12 to handle numbers beyond 11
    
    return result


In [14]:
def convert_to_year(month_num):
    # January 2013 is the starting point
    start_year = 2013
    
    # Calculate the year
    year = start_year + (month_num // 12)
    
    return year

In [15]:
a= pd.DataFrame({'a':[1,2,3]})
b=a.copy()
a['a'] +=1
print(b)

   a
0  1
1  2
2  3


In [16]:

def create_lags(data,item_lags=None,price_lags=None, test_included=False):
    #if test_included:
    #    lagged_items = data[data['date_block_num'] < 34] [['date_block_num','item_cnt_month','shop_id','item_id']].copy()
    #    lagged_prices = data[data['date_block_num'] < 34] [['date_block_num','avg_item_price','shop_id','item_id']].copy()
    #else:
    #    lagged_items = data[['date_block_num','item_cnt_month','shop_id','item_id']].copy()
    #    lagged_prices = data[['date_block_num','avg_item_price','shop_id','item_id']].copy()
    lagged_items = data[['date_block_num','item_cnt_month','shop_id','item_id']]
    lagged_prices = data[['date_block_num','avg_item_price','shop_id','item_id']]
    for lag in item_lags:
        lagged_items['date_block_num']+=lag#previous month becomes present
        data=data.merge(lagged_items, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_items['date_block_num']-=lag

    
    for lag in price_lags:
        lagged_prices['date_block_num']+=lag#previous month becomes present
        data=data.merge(lagged_prices, how='left',on=['date_block_num','shop_id','item_id'], suffixes=('', f'_lag_{lag}'))
        lagged_prices['date_block_num']-=lag

        
    return data

In [17]:
def find_mean_price(lagged):
    return lagged[[f'avg_item_price_lag_{lag}' for lag in price_lags[:6]]].mean(axis=1)
    

In [18]:
def find_mean_cnt(lagged):
    return lagged[[f'item_cnt_month_lag_{lag}' for lag in item_lags[:6]]].mean(axis=1)
    

In [19]:
def avg(data):
    v = data['item_cnt_month'].mode().mean()

    return v#.mode()[0]#.mean()

In [20]:
def avg_by_city(data, city):
    v = data [data['city']==city] ['item_cnt_month'].mode().mean()
    
    return v# .mode()[0]

In [21]:
def avg_by_shop(data, shop):
    v = data [data['shop_id']==shop] ['item_cnt_month'].mode().mean()
    
    return v# .mode()[0]#.mean()

In [22]:

def create_lags_columns(data, columns,item_lags=[1,2,3],price_lags=[1,2,3], test_included=False):#date_block_num required as first argument
    #if test_included:
    #    lagged_items = data [data['date_block_num'] < 34]  [[*columns,'item_cnt_month']] 
    #    lagged_prices = data [data['date_block_num'] < 34]  [[*columns,'avg_item_price']]
    #else:
    lagged_items = data[[*columns,'item_cnt_month']]
    lagged_prices = data[[*columns,'avg_item_price']]
    
    for lag in item_lags:
        lagged_items['date_block_num']+=lag#previous month becomes present

        averaged = lagged_items.groupby(columns).mean().reset_index()#future 
        
        data=data.merge(averaged, how='left',on=columns, suffixes=('', f'{'_'.join(columns[1:])}_lag_{lag}'))
        
        lagged_items['date_block_num']-=lag

    
    for lag in price_lags:
        lagged_prices['date_block_num']+=lag#previous month becomes present

        averaged = lagged_prices.groupby(columns).mean().reset_index()
        
        data=data.merge(averaged, how='left',on=columns, suffixes=('', f'{'_'.join(columns[1:])}_lag_{lag}'))
        lagged_prices['date_block_num']-=lag

        
    return data

In [23]:
import pandas as pd

# Assuming your DataFrame is named 'df'

# Function to calculate EMA over the lag columns
#@numba.jit(, parallel = True, nogil = True) 
def calculate_ema_3(df, min_periods=3, target='item_cnt_month', columns=None, test_included=False):#pass date_block_num as a first colummn
    # Create a temporary dataframe with the 3 lags
    
    def calculate_ema(df,group_col):#na not ignored
        #print(df.head())
        alpha=0.7
        
        lag_data = df.iloc[0][[f'{target}{'_'.join(group_col[1:])}_lag_1', f'{target}{'_'.join(group_col[1:])}_lag_2', f'{target}{'_'.join(group_col[1:])}_lag_3']]
        lag_data=lag_data.fillna(0)
        
        weights = (1 - alpha) ** np.arange(3)
        
        weighted_sum = (lag_data * weights).sum()
        
        weights[lag_data==0]=0
        
        weight_sum = weights.sum()
        if weight_sum == 0:
            return np.nan
            
        results=weighted_sum/weight_sum
        
        return results

    #print(columns)
    calcs=df.groupby(columns).apply(lambda group: calculate_ema(group, columns),include_groups=False).reset_index(name =f'ema_{target}_{'_'.join(columns[1:])}')
    #print(calcs)
    
    df = pd.merge(df, calcs, on=columns, how='left')
    return df


In [24]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [25]:
"""
find mean for last : month
(item_id)
(item_id, city)
(item_id, shop_id)
(item_category_id, shop_id)
(item_category_id, city)
"""
def find_means_last6(lagged, columns=None):#date_block_num required as first argument
    means = lagged[['item_cnt_month_lag_1', 
                 'item_cnt_month_lag_2',
                 'item_cnt_month_lag_3',
                 'item_cnt_month_lag_4', 
                 'item_cnt_month_lag_5', 
                 'item_cnt_month_lag_6'
                   ]].mean(axis=1).rename('avg')

    
   
    m_i = pd.concat([lagged[columns], means],axis=1)
    m_i = m_i.groupby(columns)['avg'].transform('mean')
    lagged[f'mean_item_cnt_6month{'_'.join(columns[1:])}'] = m_i
    return lagged


In [26]:
def create_EMAs(df,cols, test_included=True):
    df = create_lags_columns(df,columns=cols,test_included=test_included )
    start = time.time()
    df = calculate_ema_3(df, target='item_cnt_month', columns=cols,test_included=test_included )
    end = time.time()
    #df=df.drop([f'item_cnt_month{'_'.join(cols[1:])}_lag_1', f'item_cnt_month{'_'.join(cols[1:])}_lag_2', f'item_cnt_month{'_'.join(cols[1:])}_lag_3'], axis=1)
    print('item_id EMA calculated , time,s:',end-start)
    return df

In [27]:
import time

In [28]:
def transform(data_train, item_categories,items,shops, test_included=True ):
    grouped=data_train.groupby(['shop_id','item_id','date_block_num']).agg({'item_price':'mean',
                                                                    'item_cnt_day':'sum'
                                                                    })#take some (a lot)))) ) time
    
    for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                              locals().items())), key= lambda x: -x[1])[:10]:
        print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))
        
    print('1st grouping done')
    
    grouped=grouped.reset_index()
    grouped = grouped.sort_values(by='date_block_num')
    merged = grouped.merge(items, how='left').merge(item_categories, how='left').merge(shops, how='left')
    
    print('merged created')
    
    merged['month'] = np.array(number_to_month(merged['date_block_num'])).astype(np.uint8)
    merged['year'] = merged['date_block_num'].apply(convert_to_year).astype(np.uint16)
    merged['super_category'] = merged['item_category_name'].apply(lambda a: a.split()[0]).astype('category').cat.codes.astype(np.uint8)
    merged['city'] = merged['shop_name'].apply(parse_city).astype('category').cat.codes.astype(np.uint8)
    merged=merged.rename(columns={'item_price':'avg_item_price','item_cnt_day':'item_cnt_month'})

    merged['shop_id_cat'] = merged['shop_id'].astype(np.uint8)
    merged['item_id_cat'] = merged['item_id'].astype(np.uint16)
    merged['item_category_id_cat'] = merged['item_category_id'].astype(np.uint8)

    cols=['date_block_num','item_id']
    
    merged=create_EMAs(merged,cols)#168.3924057483673
    print(f'{cols} EMA calculated')
    print(merged[f'ema_item_cnt_month_{'_'.join(cols[1:])}'].isna().sum())
  
    
    cols=['date_block_num','item_id','city']#--------------VERY LONG - 996.2963778972626
    merged=create_EMAs(merged,cols)
    print(f'{cols} EMA calculated')
    
    
    cols=['date_block_num','item_category_id_cat','shop_id']#--------------
    merged=create_EMAs(merged,cols)
    print(f'{cols} EMA calculated')
    
        
    cols=['date_block_num','item_category_id_cat','city']#------------34
    merged=create_EMAs(merged,cols)
    print(f'{cols} EMA calculated')

    
    
    item_lags=list([*range(1,7),12,13,24])
    price_lags=list([*range(1,7),12,24])

    
    
    lagged = create_lags(merged,item_lags=item_lags,price_lags=price_lags,test_included=test_included )#takes a bit time
    
    print('laggs created')
    
    #lagged = encode_numerical_features(lagged, features=['item_category_id_cat', 'super_category','city'])#this part may not be used
    #df = lagged
    
    lagged = find_means_last6(lagged, ['date_block_num','item_id','shop_id',])
    lagged = find_means_last6(lagged, ['date_block_num','item_id','city'])
    lagged = find_means_last6(lagged, ['date_block_num','item_category_id','shop_id'])
    lagged = find_means_last6(lagged, ['date_block_num','item_category_id','city'])
    lagged = find_means_last6(lagged, ['date_block_num','item_category_id'])
    #df = find_means_last6(lagged)

    
    
    print('means for last 6 mothes found')
    
    lagged['first_date_block_num'] = lagged.groupby('item_id')['date_block_num'].transform('min')
    lagged['date_block_num_diff'] = lagged['date_block_num'] - lagged['first_date_block_num']
    
    
    
    
    
    
    """lagged = create_lags_columns(lagged,columns=['date_block_num','item_id','city'],test_included=test_included )
    lagged = calculate_ema_3(lagged, target='item_cnt_month', columns=['date_block_num','item_id','city'],test_included=test_included )

    
    lagged = create_lags_columns(lagged,columns=['date_block_num','item_category_id_cat','shop_id'],test_included=test_included )
    lagged = calculate_ema_3(lagged, target='item_cnt_month', columns=['date_block_num','item_category_id_cat','shop_id'],test_included=test_included )

    print('item_category_id_cat','shop_id','EMA calculated')

    lagged = create_lags_columns(df,columns=['date_block_num','item_category_id_cat','city'],test_included=test_included )
    lagged = calculate_ema_3(lagged, target='item_cnt_month', columns=['date_block_num','item_category_id_cat','city'],test_included=test_included )
    
    print('item_category_id_cat','city','EMA calculated')"""

    
    return lagged[lagged['date_block_num'] >= 3 ]
    #to delete - lag_2,lag_3, lags for columns
            

In [29]:
#data_train.memory_usage() / (10**6)

In [30]:
test = pd.read_csv('../data_cleaned/test.csv')

In [31]:
test['date_block_num'] = 34

In [32]:
transformed = transform(pd.concat([data_train,test]),  item_categories,items,shops,test_included=True )

                    data_train: 339.2 MiB
                       grouped: 34.9 MiB
                         items:  3.3 MiB
               item_categories:  9.9 KiB
                         shops:  6.9 KiB
                 test_included:   28.0 B
1st grouping done
merged created


/tmp/ipykernel_82369/2660291697.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']+=lag#previous month becomes present
/tmp/ipykernel_82369/2660291697.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lagged_items['date_block_num']-=lag
/tmp/ipykernel_82369/2660291697.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

KeyboardInterrupt: 

In [47]:
#transformed[['item_cnt_monthitem_category_id_cat_city_lag_1',
#       'item_cnt_monthitem_category_id_cat_city_lag_2',
#       'item_cnt_monthitem_category_id_cat_city_lag_3','ema_item_cnt_month_item_category_id_cat_city']]

,item_cnt_monthitem_category_id_cat_city_lag_1,item_cnt_monthitem_category_id_cat_city_lag_2,item_cnt_monthitem_category_id_cat_city_lag_3,ema_item_cnt_month_item_category_id_cat_city
187032,1.696312,1.705607,1.567265,1.689963
187033,7.774194,6.050000,3.171053,7.104020
187034,1.329670,1.242268,1.276543,1.307367
187035,2.458333,2.325252,2.231736,2.414939
187036,2.458333,2.325252,2.231736,2.414939
...,...,...,...,...
1822420,NaN,NaN,NaN,NaN
1822421,1.000000,2.000000,NaN,1.230769
1822422,1.195122,1.576923,1.225806,1.279512
1822423,1.000000,1.000000,1.000000,1.000000


In [44]:
transformed.columns

Index(['shop_id', 'item_id', 'date_block_num', 'avg_item_price',
       'item_cnt_month', 'item_name', 'item_category_id', 'item_category_name',
       'shop_name', 'month', 'year', 'super_category', 'city', 'shop_id_cat',
       'item_id_cat', 'item_category_id_cat', 'item_cnt_monthitem_id_lag_1',
       'item_cnt_monthitem_id_lag_2', 'item_cnt_monthitem_id_lag_3',
       'avg_item_priceitem_id_lag_1', 'avg_item_priceitem_id_lag_2',
       'avg_item_priceitem_id_lag_3', 'ema_item_cnt_month_item_id',
       'item_cnt_monthitem_id_city_lag_1', 'item_cnt_monthitem_id_city_lag_2',
       'item_cnt_monthitem_id_city_lag_3', 'avg_item_priceitem_id_city_lag_1',
       'avg_item_priceitem_id_city_lag_2', 'avg_item_priceitem_id_city_lag_3',
       'ema_item_cnt_month_item_id_city',
       'item_cnt_monthitem_category_id_cat_shop_id_lag_1',
       'item_cnt_monthitem_category_id_cat_shop_id_lag_2',
       'item_cnt_monthitem_category_id_cat_shop_id_lag_3',
       'avg_item_priceitem_category_id

In [45]:
transformed['ema_item_cnt_month_item_category_id_cat_city'].isna().sum()

np.int64(32312)

In [46]:
def select_features(data):
    data=self.data [[
        
    ]]
    return data

In [47]:
#test_trans=select_features(transformed)

In [48]:
test_trans=transformed

In [49]:
list(test_trans.columns)


['shop_id',
 'item_id',
 'date_block_num',
 'avg_item_price',
 'item_cnt_month',
 'item_name',
 'item_category_id',
 'item_category_name',
 'shop_name',
 'month',
 'year',
 'super_category',
 'city',
 'shop_id_cat',
 'item_id_cat',
 'item_category_id_cat',
 'item_cnt_monthitem_id_lag_1',
 'item_cnt_monthitem_id_lag_2',
 'item_cnt_monthitem_id_lag_3',
 'avg_item_priceitem_id_lag_1',
 'avg_item_priceitem_id_lag_2',
 'avg_item_priceitem_id_lag_3',
 'ema_item_cnt_month_item_id',
 'item_cnt_monthitem_id_city_lag_1',
 'item_cnt_monthitem_id_city_lag_2',
 'item_cnt_monthitem_id_city_lag_3',
 'avg_item_priceitem_id_city_lag_1',
 'avg_item_priceitem_id_city_lag_2',
 'avg_item_priceitem_id_city_lag_3',
 'ema_item_cnt_month_item_id_city',
 'item_cnt_monthitem_category_id_cat_shop_id_lag_1',
 'item_cnt_monthitem_category_id_cat_shop_id_lag_2',
 'item_cnt_monthitem_category_id_cat_shop_id_lag_3',
 'avg_item_priceitem_category_id_cat_shop_id_lag_1',
 'avg_item_priceitem_category_id_cat_shop_id_lag_2

In [51]:
test_trans.to_csv('../data.csv')

In [28]:
test_trans.query('date_block_num==34')

,shop_id,item_id,date_block_num,avg_item_price,item_cnt_month,item_name,item_category_id,item_category_name,shop_name,month,...,item_category_id_cat,date_block_num_diff,ema_avg_item_price_item_category_name,ema_avg_item_price_item_id,ema_avg_item_price_super_category,ema_avg_item_price_city,ema_item_cnt_month_item_category_name,ema_item_cnt_month_item_id,ema_item_cnt_month_super_category,ema_item_cnt_month_city
1608225,59,22166,34,NaN,0.0,Язык запросов 1С:Предприятия [Цифровая версия],54,Книги - Цифра,"Ярославль ТЦ ""Альтаир""",10,...,54,25,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608226,59,22164,34,NaN,0.0,ЯРОСТЬ (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир""",10,...,37,9,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608227,59,22163,34,NaN,0.0,ЯРОСТЬ ( регион),40,Кино - DVD,"Ярославль ТЦ ""Альтаир""",10,...,40,3,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608228,59,22162,34,NaN,0.0,ЯРОСТЬ,40,Кино - DVD,"Ярославль ТЦ ""Альтаир""",10,...,40,16,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
1608229,2,32,34,NaN,0.0,1+1,40,Кино - DVD,"Адыгея ТЦ ""Мега""",10,...,40,34,1074.836301,NaN,762.896909,936.470404,1.541233,NaN,3.535836,2.992044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822420,2,42,34,NaN,0.0,100 Best romantic melodies (mp3-CD) (Digipack),57,Музыка - MP3,"Адыгея ТЦ ""Мега""",10,...,57,34,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493
1822421,2,38,34,NaN,0.0,10 САМЫХ ПОПУЛЯРНЫХ КИНОКОМЕДИЙ ХХ ВЕКА 10DVD...,41,Кино - Коллекционное,"Адыгея ТЦ ""Мега""",10,...,41,16,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493
1822422,2,33,34,NaN,0.0,1+1 (BD),37,Кино - Blu-Ray,"Адыгея ТЦ ""Мега""",10,...,37,34,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493
1822423,59,22167,34,NaN,0.0,Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,49,Книги - Методические материалы 1С,"Ярославль ТЦ ""Альтаир""",10,...,49,26,139.127975,NaN,139.127975,990.606128,3.014701,NaN,3.014701,1.930493


In [30]:
test_trans.query('date_block_num==34') .isna().sum()

shop_id                                       0
item_id                                       0
date_block_num                                0
avg_item_price                           214200
item_cnt_month                                0
item_name                                     0
item_category_id                              0
item_category_name                            0
shop_name                                     0
month                                         0
year                                          0
super_category                                0
city                                          0
item_cnt_month_lag_1                     185528
item_cnt_month_lag_2                     187735
item_cnt_month_lag_3                     185251
item_cnt_month_lag_4                     187761
item_cnt_month_lag_5                     189988
item_cnt_month_lag_6                     191029
item_cnt_month_lag_12                    191776
item_cnt_month_lag_13                   

In [29]:
#set_y = set(map(tuple, test_trans.query('date_block_num==33') [['shop_id','item_id']].values))
#idx = [tuple(point) not in set_y for point in test_trans.query('date_block_num==34') [['shop_id','item_id']].values]

#test_trans.query('date_block_num==34') [['shop_id','item_id']].values[idx].shape

In [27]:
!pwd

/home/valeriy/python_projects/predict_future_sales/EDA
